<a href="https://colab.research.google.com/github/AtharvBhat/PIAYN/blob/main/PIAYN_TextLRABaseline_3k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set-up environment

As usual, we first install HuggingFace Transformers, and Datasets.

In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [2]:
!pip install -q datasets

In [3]:
!pip install ml_collections
!pip install absl-py

In [4]:
!git clone https://github.com/google-research/long-range-arena.git
!git clone https://github.com/mlpen/Nystromformer.git
!git clone https://github.com/AtharvBhat/PIAYN.git

fatal: destination path 'long-range-arena' already exists and is not an empty directory.
fatal: destination path 'Nystromformer' already exists and is not an empty directory.
fatal: destination path 'PIAYN' already exists and is not an empty directory.


## Set up output folders

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#Set the task
task = 'text'

#Set Seq Length
max_length = 3000

#Set Seed
seed = 42

In [7]:
import os

baseDir = "/content/drive/MyDrive"
piaynDir = os.path.join(baseDir, "PIAYN")

if not os.path.exists(piaynDir):
  os.makedirs(piaynDir)

piaynTaskDir = os.path.join(piaynDir, task)

if not os.path.exists(piaynTaskDir):
  os.makedirs(piaynTaskDir)

piaynTaskModelDir = os.path.join(piaynTaskDir, "models")

if not os.path.exists(piaynTaskModelDir):
  os.makedirs(piaynTaskModelDir)

piaynTaskDataDir = os.path.join(piaynTaskDir, "data")

if not os.path.exists(piaynTaskDataDir):
  os.makedirs(piaynTaskDataDir)    

piaynTaskDataDirTokens = os.path.join(piaynTaskDataDir, str(max_length))

if not os.path.exists(piaynTaskDataDirTokens):
  os.makedirs(piaynTaskDataDirTokens) 

piaynTaskModelDirTokens = os.path.join(piaynTaskModelDir, str(max_length))

if not os.path.exists(piaynTaskModelDirTokens):
  os.makedirs(piaynTaskModelDirTokens)  

## Prepare data

Here we take a small portion of the IMDB dataset, a binary text classification dataset ("is a movie review positive or negative?").

In [8]:
from datasets import load_dataset
#from Nystromformer.LRA.datasets import text
import pickle, numpy as np
import torch
import random

#Add all seeds to make it deterministic
if seed is not None:
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  np.random.seed(seed)
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)

#copy over the custom text.py file to the Nystromformer location
!cp PIAYN/text.py Nystromformer/LRA/datasets

#Create Train test and dev MAP files for the imdb dataset
!python Nystromformer/LRA/datasets/text.py --max_length=$max_length

train_file = task+'.train.pickle'
dev_file = task+'.dev.pickle'
test_file = task+'.test.pickle'

!cp $train_file $piaynTaskDataDirTokens
!cp $dev_file $piaynTaskDataDirTokens
!cp $test_file $piaynTaskDataDirTokens

max_length:  3000
I0415 02:19:09.706180 140362176395136 dataset_builder.py:811] No config specified, defaulting to first: imdb_reviews/plain_text
I0415 02:19:09.707057 140362176395136 dataset_info.py:361] Load dataset info from /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0
I0415 02:19:09.709865 140362176395136 dataset_builder.py:299] Reusing dataset imdb_reviews (/root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0)
I0415 02:19:09.710001 140362176395136 dataset_builder.py:511] Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0
2022-04-15 02:19:10.898373: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0415 02:19:11.132364 140362176395136 input_pipeline.py:49] Data sample: {'label': 1, 'text': b'As others have mentioned, all the women that go nude in this film are mostly absolute

In [9]:
from Nystromformer.LRA.code import lra_config
from Nystromformer.LRA.code.dataset import LRADataset
#from Nystromformer.LRA.code.run_tasks import training_config
from torch.utils.data import DataLoader, RandomSampler

#get training config
training_config = lra_config.config[task]["training"]

#Update training config
#training_config["learning_rate"] = 0.05
training_config["weight_decay"] = 0.1
#training_config["eval_frequency"] = 1000

#Check Train Config
print('Training Config: ', training_config)

#get pre-defined model config
model_config = lra_config.config[task]['model']

#Check model Config
print('Model Config: ', model_config)

#Get the dataset
train_dataset = LRADataset(piaynTaskDataDirTokens + '/' + train_file, False)
val_dataset = LRADataset(piaynTaskDataDirTokens + '/' + dev_file, False)
test_dataset = LRADataset(piaynTaskDataDirTokens + '/' + test_file, False)

#Create DataLoader iterators
ds_iter = {
    "train":enumerate(DataLoader(train_dataset, 
                                 #Sample batches randomly for number of specified steps
                                 sampler = RandomSampler(train_dataset, 
                                                         replacement=True, 
                                                         num_samples= training_config["num_train_steps"]*lra_config.config[task]['dataset']['train']), 
                                 batch_size = training_config["batch_size"], 
                                 drop_last = True)),
    "dev":enumerate(DataLoader(val_dataset, batch_size = 32, drop_last = True)),
    "test":enumerate(DataLoader(test_dataset, batch_size = 32, drop_last = True)),
}


Training Config:  {'batch_size': 32, 'learning_rate': 0.0001, 'warmup': 8000, 'lr_decay': 'linear', 'weight_decay': 0.1, 'eval_frequency': 500, 'num_train_steps': 20000, 'num_eval_steps': 781}
Model Config:  {'learn_pos_emb': True, 'tied_weights': False, 'embedding_dim': 64, 'transformer_dim': 64, 'transformer_hidden_dim': 128, 'head_dim': 32, 'num_head': 2, 'num_layers': 2, 'vocab_size': 512, 'max_seq_len': 4000, 'dropout_prob': 0.1, 'attention_dropout': 0.1, 'pooling_mode': 'MEAN', 'num_classes': 2}
Loaded /content/drive/MyDrive/PIAYN/text/data/3000/text.train.pickle... size=25000
Loaded /content/drive/MyDrive/PIAYN/text/data/3000/text.dev.pickle... size=25000
Loaded /content/drive/MyDrive/PIAYN/text/data/3000/text.test.pickle... size=25000


In [10]:
#Check sizes of batches
batch = next((ds_iter['train']))
for k,v in batch[1].items():
  print(k,v.shape)

input_ids_0 torch.Size([32, 3024])
mask_0 torch.Size([32, 3024])
label torch.Size([32])


## Define model

Next, we define our model, and put it on the GPU.

In [11]:
from transformers import PerceiverForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
from transformers import PerceiverConfig
#get default perceiver config
configuration = PerceiverConfig()

#Update the Perceiver configurations with Preset model configs
#configuration.update(model_config)

#Print Updated Perceiver Configuration
print(configuration)

PerceiverConfig {
  "attention_probs_dropout_prob": 0.1,
  "audio_samples_per_frame": 1920,
  "cross_attention_shape_for_attention": "kv",
  "cross_attention_widening_factor": 1,
  "d_latents": 1280,
  "d_model": 768,
  "hidden_act": "gelu",
  "image_size": 56,
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "perceiver",
  "num_blocks": 1,
  "num_cross_attention_heads": 8,
  "num_frames": 16,
  "num_latents": 256,
  "num_self_attends_per_block": 26,
  "num_self_attention_heads": 8,
  "output_shape": [
    1,
    16,
    224,
    224
  ],
  "qk_channels": null,
  "samples_per_patch": 16,
  "self_attention_widening_factor": 1,
  "train_size": [
    368,
    496
  ],
  "transformers_version": "4.19.0.dev0",
  "use_query_residual": true,
  "v_channels": null,
  "vocab_size": 262
}



In [13]:
def initialize_model(config):
  #Initialize Model
  model = PerceiverForSequenceClassification(config)
  
  #Get Model Parameter Counts
  pytorch_total_params = sum(p.numel() for p in model.parameters())
  pytorch_total_params_Trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
  print('Total Parameters: ', pytorch_total_params, '\nTrainable Parameters: ', pytorch_total_params_Trainable)  

  return model#.to(device)                                          

In [14]:
#Change the perceiver configurations to get total parameters within 10% of BERT
configuration.num_labels = 2
configuration.num_self_attends_per_block = 6
configuration.d_latents = 512
configuration.d_model = 512
configuration.num_layers = 6
if max_length>2000:
  configuration.max_position_embeddings = max_length + 24*np.power(2, int(max_length/1000)-1)



model = initialize_model(configuration)

Total Parameters:  14477826 
Trainable Parameters:  14477826


## Train the model

Here we train the model using native PyTorch.

In [15]:
device

device(type='cuda')

In [16]:
from transformers import AdamW
#from torch.optim import Adam
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from datasets import load_metric
import pandas as pd

best_score = 0 
prev_score = 0
maxPatience = 5
currentPatience = 0

#steps = int(training_config["num_train_steps"]/20000)
steps = training_config["num_train_steps"]

optimizer = AdamW(model.parameters(), 
                  lr = training_config["learning_rate"],
                  betas = (0.9, 0.999), 
                  eps = 1e-6, 
                  weight_decay = training_config["weight_decay"])

lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer = optimizer,
    max_lr = training_config["learning_rate"],
    pct_start = training_config["warmup"] / training_config["num_train_steps"],
    anneal_strategy = training_config["lr_decay"],
    total_steps = training_config["num_train_steps"]
)

#amp_scaler = torch.cuda.amp.GradScaler() if model_config["mixed_precision"] else None

#initialize training summary
trainingSummary = pd.DataFrame(columns=['step', 'mean_train_loss', 'mean_train_acc', 'val_acc'])


model.to(device)

#initialize training accuracy metric and loss list
train_accuracy = load_metric("accuracy")
loss_list = list()

for step in tqdm(range(steps)):  # Perform gradient updates for multiple steps
    
    model.train()
    
    #print("Step:", step)
    #for batch in tqdm(train_dataloader):
    batch = next(ds_iter['train'])[1]

    # get the inputs; 
    inputs = batch["input_ids_0"].to(device)
    attention_mask = batch["mask_0"].to(device)
    labels = batch["label"].to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs=inputs, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    lr_scheduler.step()

    # evaluate
    predictions = outputs.logits.argmax(-1).cpu().detach().numpy()
    accuracy = accuracy_score(y_true=batch["label"].numpy(), y_pred=predictions)
    references = batch["label"].numpy()
    train_accuracy.add_batch(predictions=predictions, references=references)
    
    #Add to loss list
    loss_list.append(loss.item())

    #print(f"Loss: {loss.item()}, Accuracy: {accuracy}")

    #delete intermediate variables to free up GPU space
    del loss, outputs, inputs, attention_mask, labels, predictions, accuracy


    #Every 1000 steps validate and save model
    if (step+1)%training_config['eval_frequency']  == 0:
    #if (step+1)%2  == 0:
      
      model.eval()

      print('Validating at Step: ', step)

      val_accuracy = load_metric("accuracy")

      #reset dev iterator
      ds_iter['dev'] = enumerate(DataLoader(val_dataset, batch_size = 32, drop_last = True))

      with torch.no_grad():
        for i, batch in tqdm(ds_iter['dev']):
              
          # get the inputs; 
          inputs = batch["input_ids_0"].to(device)
          attention_mask = batch["mask_0"].to(device)
          labels = batch["label"].to(device)

          # forward pass
          outputs = model(inputs=inputs, attention_mask=attention_mask)
          logits = outputs.logits 
          predictions = logits.argmax(-1).cpu().detach().numpy()
          references = batch["label"].numpy()
          val_accuracy.add_batch(predictions=predictions, references=references)

          #delete intermediate variables to free up GPU space
          del logits, outputs, inputs, attention_mask, labels, predictions, references
      
      #Compute val accuracy
      final_val_score = val_accuracy.compute()['accuracy']

      #Compute training accuracy till now
      train_score = train_accuracy.compute()['accuracy']

      #Compute training loss till now
      train_loss = sum(loss_list)/len(loss_list)

      #Add to trainingSummary
      trainingSummary.loc[len(trainingSummary.index)] = [step+1, train_loss, train_score, final_val_score]

      #save training summary
      trainingSummary.to_csv(piaynTaskModelDirTokens + '/trainingSummaryToken'+str(max_length)+'.csv')

      #print progress
      print('Step: ', step+1, "\n\tAverage Train Loss: ", train_loss, "\n\tAverage Train Accuracy: ", train_score, "\n\tValidation Accuracy: ", final_val_score)

      #Save if performance better than best model
      if final_val_score >= best_score:
        best_score = final_val_score
        torch.save(model.to('cpu').state_dict(), piaynTaskModelDirTokens + '/trainedPerceiverClassifierToken'+str(max_length)+'.pkl')
        model.to(device)
      else:
        pass  

      #Stop training if patience limit reached
      if final_val_score <= prev_score:
        currentPatience += 1
        if currentPatience >= maxPatience:
          print('Patience Limit reached! Stopping early!')
          torch.save(model.to('cpu').state_dict(), piaynTaskModelDirTokens + '/trainedPerceiverClassifierStep_' + str(step + 1) + 'Token' + str(max_length) + '.pkl')
          break  
      else:
        currentPatience = 0
      
      #Update prev_score
      prev_score = final_val_score

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/20000 [00:00<?, ?it/s]

Validating at Step:  499


0it [00:00, ?it/s]

Step:  500 
	Average Train Loss:  0.6783526061773301 
	Average Train Accuracy:  0.559625 
	Validation Accuracy:  0.569182138284251
Validating at Step:  999


...

Step:  19000 
	Average Train Loss:  0.39886368110994075 
	Average Train Accuracy:  0.979375 
	Validation Accuracy:  0.5977512804097311
Validating at Step:  19499


0it [00:00, ?it/s]

Step:  19500 
	Average Train Loss:  0.39040530609413504 
	Average Train Accuracy:  0.9813125 
	Validation Accuracy:  0.6078745198463509
Validating at Step:  19999


0it [00:00, ?it/s]

Step:  20000 
	Average Train Loss:  0.38242311142771507 
	Average Train Accuracy:  0.98125 
	Validation Accuracy:  0.6073943661971831


In [17]:
!nvidia-smi

Fri Apr 15 04:54:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    51W / 250W |   8697MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
del model
torch.cuda.empty_cache()
!nvidia-smi

Fri Apr 15 04:54:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    53W / 250W |   1371MiB / 16280MiB |     96%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Evaluate the model

Finally, we evaluate the model on the test set. We use the Datasets library to compute the accuracy.

In [19]:
from tqdm.notebook import tqdm
from datasets import load_metric

accuracy = load_metric("accuracy")

#load best performing model checkpoint
model = PerceiverForSequenceClassification(configuration)
model.load_state_dict(torch.load(piaynTaskModelDirTokens + '/trainedPerceiverClassifierToken'+str(max_length)+'.pkl'))
model.to(device)

model.eval()

with torch.no_grad():
  for i, batch in tqdm(ds_iter['test']):
        
        # get the inputs; 
        inputs = batch["input_ids_0"].to(device)
        attention_mask = batch["mask_0"].to(device)
        labels = batch["label"].to(device)

        # forward pass
        outputs = model(inputs=inputs, attention_mask=attention_mask)
        logits = outputs.logits 
        predictions = logits.argmax(-1).cpu().detach().numpy()
        references = batch["label"].numpy()
        accuracy.add_batch(predictions=predictions, references=references)

        #delete intermediate variables to free up GPU space
        del logits, outputs, inputs, attention_mask, labels, predictions, references

final_score = accuracy.compute()
print("Accuracy on test set:", final_score['accuracy'])

0it [00:00, ?it/s]

Accuracy on test set: 0.6493677976952625
